# Performance of Directors‘ Dealings transactions in shares of Nasdaq 100 companies pre vs during Covid 19 Pandemic – 2018 - 2021
Interdisciplinary Project 2021W? 2022S?
Gunnar S. Knudsen

Advisor:  Wolfgang Aussenegg

## Description

Here I should write stuff about what this notebook does

## Own Notes

#### Questions for Professor Aussenegg
* What is expected of me?
* Do I need a second advisor?
* Regular meetups?
* 125 hours in total
* **What is the problem statement?** - Defined in TUWEL what it needs to contain. 
* What is final handin?
  * Report (How long?)
  * Publication? (Far fetched goal?)
  * Poster!?
* From task description:
  * what is "PEF_8 pages 32-40 as well as the start literature"
* Implementation specific questions:
  * what is `delta_Own` and `TradeType`? How to handle?
  * Is there no insider transaction for some? E.g. http://openinsider.com/screener?s=ASML&o=&pl=&ph=&ll=&lh=&fd=0&fdr=&td=0&tdr=&fdlyl=&fdlyh=&daysago=&xp=1&xs=1&vl=&vh=&ocl=&och=&sic1=-1&sicl=100&sich=9999&grp=0&nfl=&nfh=&nil=&nih=&nol=&noh=&v2l=&v2h=&oc2l=&oc2h=&sortcol=0&cnt=100&page=1 ?
  * What is Form 3 and 5? Should this be considered? - NVM; I can maybe read up on this


#### Questions for me
* Consider Form 3 and 5?
* Is there a diffefrence between issuer and reporter?

#### Todo
  * Figure out what the parameters are in the queries
  * store results per ticker as csv
  * read in csvs
  * get newest date for ticker
  * pass date into fetch
  * run loop until all is fetched - Can be checked with less than 1000 rows fetched
  * how to handle duplicates?
  * save to csv per ticker
  * concatenate
  * fix spaces in header with `string = string.replace(u'\xa0', u' ')`
  * understand and fix ΔOwn
  * add a flag for (re)download shit
  * figure out why some Tickers just break
  * combine if multiple insider trades occur during the same day
  * Title and TradeType - Rework this in some way!
  
#### Relevant links
* https://www.investopedia.com/terms/a/abnormalreturn.asp
* https://codingandfun.com/analysing-inside-trading-within-a-company-with-python/
* https://medium.com/@jan_5421/insider-trading-visualised-with-python-sec-api-io-5f12a7799b3e - Do this!
* https://medium.com/automated-trading/how-to-get-insider-trading-data-using-python-f93bf552e88e
* https://www.reddit.com/r/market_sentiment/comments/pr7i7e/should_you_follow_insider_transactions_i_analyzed/
* https://medium.com/wealthy-bytes/visualizing-free-stock-data-for-algorithmic-trading-with-python-and-matplotlib-dca1abbd286c
* https://www.analyticsvidhya.com/blog/2021/07/stock-prices-analysis-with-python/ Looks promising

### Badly written notes from 2021-12-20

ar vs car inmckinley 

significant test

compare with found in literature


Deliver data - DD and how I calculated AR - analyse cost section CAR
Matlab? R?

Report! Seminar paper What I've done, why is this interesting from point of literature view. What are the results

spend time on several DDs for one company - this is time realted.
Take the filing date. Trading day can be 1-2 days before.
Information symmetry! is the reason. Should be measureable.

Report: 15+/- more than 30 is too much and less than 10.

Would need to work more

after 40 

writing why no insiders are for some
descriptive statistics

stock returns

describe for 8/9 companies

form 4 is insider trading information

owned is owner prior to qty

SEC has website, could be an alternative!

delta_own: is an error!?
Sale + OE: see website!
They get options as part of compentsaion. As this is part of income, they are regularly selling shares, this type of trades could thus be less informative. 
Purchase should be more infromative
Sale should be less informative

compare types of trades

A lot of other factos influence prizes - Large enough portfolio, all the other information should be diversitfied away. 


Need an index for accounting/calculating market model. Expected returns based also on the risk analysis of the company.


13. of january
Compare the realized returns with abnormal returns


3-5 deal with equity - rest is not relevant

I write proposal, send to professors
He sends me reading material


## Setup
Libraries and constants

### Required Libraries

In [1]:
# Standard
import pandas as pd
import numpy as np
import pickle

# Time Cleaning
import time
import datetime
from dateutil.relativedelta import relativedelta

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

# Scraping
import requests
import locale
from pandas.io.json import json_normalize


from os.path import exists


### Parameter Configurations

In [2]:
locale.setlocale(locale.LC_ALL,'')
sns.set_theme(style="ticks")

### Constants

#### Flags
Don't want to send unnecesary requests when testing notebook

In [3]:
REDOWNLOAD_TICKERS = True
REDOWNLOAD_INSIDER_DATA = True
REPROCESS_INSIDER_DATA = True
REDOWNLOAD_TIMESERIES_DATA = True

#### Constants

In [4]:
DATA_LOCATION = 'data/'

DATA_LOCATION_INSIDER_RAW = 'data/raw/insider/'
DATA_LOCATION_INSIDER_PROCESSED = 'data/processed/insider/'

DATA_LOCATION_TIME_SERIES_RAW = 'data/raw/timeseries/'
DATA_LOCATION_TIME_SERIES_PROCESSED = 'data/processed/timeseries/'

#### Helpers 
Will probably be moved to separate files later

In [5]:
def visualize_price_moving_average(ticker, ma1, ma2):
    df = timeseries[timeseries.ticker == ticker].copy()

    ticker_data = pd.DataFrame.from_dict(main_data)
    company_name = ticker_data[ticker_data.symbol == ticker]['companyName'].values[0]

    trace1 = {
        'x': df.Date,
        'open': df.Open,
        'close': df.Close,
        'high': df.High,
        'low': df.Low,
        'type': 'candlestick',
        'name': ticker,
        'showlegend': True
    }

    avg_30 = df.Close.rolling(window=ma1, min_periods=1).mean()
    avg_50 = df.Close.rolling(window=ma2, min_periods=1).mean()

    trace2 = {
        'x': df.Date,
        'y': avg_30,
        'type': 'scatter',
        'mode': 'lines',
        'line': {
            'width': 1,
            'color': 'blue'
                },
        'name': 'Moving Average of ' + str(ma1) + ' periods'
    }

    trace3 = {
        'x': df.Date,
        'y': avg_50,
        'type': 'scatter',
        'mode': 'lines',
        'line': {
            'width': 1,
            'color': 'red'
        },
        'name': 'Moving Average of ' + str(ma2) + ' periods'
    }

    data = [trace1, trace2, trace3]
    # Config graph layout
    layout = go.Layout({
        'title': {
            'text': company_name + '('+ticker+') Moving Averages',
            'font': {
                'size': 15
            }
        }
    })


    fig = go.Figure(data=data, layout=layout)
    fig.write_html(company_name + '('+ticker+') Moving Averages.html')
    fig.show()

In [6]:
def visualize_price_bollinger_bands(ticker, ma_size = 20, n_stds = 2):
    df = timeseries[timeseries.ticker == ticker].copy()

    ticker_data = pd.DataFrame.from_dict(main_data)
    company_name = ticker_data[ticker_data.symbol == ticker]['companyName'].values[0]

    middle_band = df.Close.rolling(window=ma_size, min_periods=1).mean()
    std_size = df.Close.rolling(window=ma_size, min_periods=1).std()
    upper_band = middle_band + n_stds * std_size
    lower_band = middle_band - n_stds * std_size

    trace_true = {
        'x': df.Date,
        'open': df.Open,
        'close': df.Close,
        'high': df.High,
        'low': df.Low,
        'type': 'candlestick',
        'name': 'Close',
        'showlegend': True
    }
    
    trace_middle = {
        'x': df.Date,
        'y': middle_band,
        'type': 'scatter',
        'mode': 'lines',
        'line': {
            'width': 1,
            'color': 'blue'
                },
        'name': 'middle'
    }
    
    trace_upper = {
        'x': df.Date,
        'y': upper_band,
        'type': 'scatter',
        'mode': 'lines',
        'line': {
            'width': 1,
            'color': 'green'
                },
        'name': 'upper'
    }
    
    trace_lower = {
        'x': df.Date,
        'y': lower_band,
        'type': 'scatter',
        'mode': 'lines',
        'line': {
            'width': 1,
            'color': 'yellow'
                },
        'name': 'lower'
    }


    data = [trace_true, trace_middle, trace_upper, trace_lower]
    # Config graph layout
    layout = go.Layout({
        'title': {
            'text': company_name + '('+ticker+') Bollinger bands of size ' + str(ma_size),
            'font': {
                'size': 15
            }
        }
    })


    fig = go.Figure(data=data, layout=layout)
    fig.write_html(company_name + '('+ticker+') Bollinger_Bands.html')
    fig.show()

## Get NASDAQ100 Components
First step is to get the tickers of the companies that are part of the NASDAQ100

In [7]:
if REDOWNLOAD_TICKERS:
    # Run request against nasdaq
    headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36"}
    res=requests.get("https://api.nasdaq.com/api/quote/list-type/nasdaq100",headers=headers)
    main_data=res.json()['data']['data']['rows']
    print("Downloaded NASDAQ information")

    # Gather list of tickers
    tickers = []
    for i in range(len(main_data)):
        #print(f"{main_data[i]['symbol']}: {main_data[i]['companyName']}")
        tickers.append(main_data[i]['symbol'])

    # Save list to file
    with open(DATA_LOCATION + 'tickers.pickle', 'wb') as handle:
        pickle.dump(tickers, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("Saved tickers to file")
    
    with open(DATA_LOCATION + 'tickers_main_data.pickle', 'wb') as handle:
        pickle.dump(main_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print("Saved main_data to file")

# Load downloaded file
with open(DATA_LOCATION + 'tickers.pickle', 'rb') as handle:
    tickers = pickle.load(handle)
print("Loaded tickers to file")    

with open(DATA_LOCATION + 'tickers_main_data.pickle', 'rb') as handle:
    main_data = pickle.load(handle)
print("Loaded main_data to file")    

Downloaded NASDAQ information
Saved tickers to file
Saved main_data to file
Loaded tickers to file
Loaded main_data to file


## Insider Transactions
It contains a summary of SEC Form 4 Filings of (legal) insider transactions.
This section will download and process the data 

### Read bigger list from Prof

In [8]:
NASDAQ_COMPOSITE = pd.read_excel('data/Nasdaq Composite 16.3.2022 plus dead firms - Knudsen.xlsx')
tickers = NASDAQ_COMPOSITE['TICKER SYMBOL'].astype(str).replace(' ', '+')

tickers

0       VCXA
1        TXG
2         YI
3         YQ
4       ATNF
        ... 
4073    ZUMZ
4074    CNET
4075      ZY
4076    ZYNE
4077    ZNGA
Name: TICKER SYMBOL, Length: 4078, dtype: object

In [9]:
'str a'.replace(' ', '+')

'str+a'

### Download raw data for insider trading
This just downloads the table as-is from openinsider.com. Will be processed at a later step.

In [10]:
if REDOWNLOAD_INSIDER_DATA:
    for ticker in tickers:
        #print(ticker)
        ticker = ticker.replace(' ', '+')
        f = DATA_LOCATION_INSIDER_RAW + ticker + '.csv'
        if not exists(f):
            insider_data = pd.read_html(f'http://www.openinsider.com/screener?s={ticker}&o=&pl=&ph=&ll=&lh=&fd=1461&fdr=&td=1461&tdr=&fdlyl=&fdlyh=&daysago=&xp=1&xs=1&vl=&vh=&ocl=&och=&sic1=-1&sicl=100&sich=9999&grp=0&nfl=&nfh=&nil=&nih=&nol=&noh=&v2l=&v2h=&oc2l=&oc2h=&sortcol=0&cnt=1000&page=1')
            insider_data = insider_data[-3]
            insider_data['ticker'] = ticker
            if (insider_data.shape[0] >= 999):
                print(f"POSSIBLE ERROR OCCURED HERE for {ticker}")

            # Rewrite if nothing is found
            if insider_data.iloc[0,0] == 'Sort by':
                insider_data = pd.DataFrame(columns=['X', 'Filing\xa0Date', 'Trade\xa0Date', 'Ticker', 'Insider Name', 'Title','Trade Type', 'Price', 'Qty', 'Owned', 'ΔOwn', 'Value', '1d', '1w','1m', '6m', 'ticker'])
            insider_data.to_csv(DATA_LOCATION_INSIDER_RAW + ticker + '.csv')

### Process data
This step is still somewhat broken

In [11]:
#tickers = ['GOBI']

if REPROCESS_INSIDER_DATA:
    for ticker in tickers:
        ticker = ticker.replace(' ', '+')
        #print(ticker)
        ## Poor mans exception handling:
        #if ticker not in ['ASML', 'TEAM', 'BIDU', 'CHKP', 'FOXA', 'JD', 'NTES', 'PDD', 'TCOM']:
        # Read in the downloaded data
        insider_data = pd.read_csv(DATA_LOCATION_INSIDER_RAW + ticker + '.csv', index_col=0)

        # Process datatypes accordingly
        insider_data['Filing\xa0Date'] = pd.to_datetime(insider_data['Filing\xa0Date'])
        insider_data['Trade\xa0Date'] = pd.to_datetime(insider_data['Trade\xa0Date'])
        ## 'Trade\xa0Type' - should this be decoded?
        insider_data['Price'] = insider_data['Price'].astype(str).map(lambda x: x.replace(',', '').strip('+'))
        insider_data['Price']=insider_data['Price'].map(lambda x: locale.atof(x.strip('$')))
        insider_data['Qty'] = pd.to_numeric((insider_data['Qty']).astype(str).map(lambda x: x.replace(',', '').strip('+')))
        insider_data['Value']=pd.to_numeric(insider_data['Value'].map(lambda x: locale.atof(x.replace(',', '').replace('$',''))))
        #insider_data['ΔOwn']=insider_data['ΔOwn'].map(lambda x: locale.atof(x.replace(',', '').replace('%','')))

        # Better naming of columns
        insider_data.columns = insider_data.columns.map(lambda x: x.replace('\xa0', '').replace('Δ', 'delta_'))
        insider_data.to_csv(DATA_LOCATION_INSIDER_PROCESSED + ticker + '.csv')

        # Usefull helpers for when I finish this step
        #insider_data
        #print(insider_data.columns.tolist())
        #insider_data.dtypes
        
        #display(insider_data)

### Combine into a single dataframe
Create a single dataframe to contain all the insider transactions

In [12]:
# Empty dummy
insider = pd.DataFrame()

for ticker in tickers:
    ticker = ticker.replace(' ', '+')
    print(ticker)
    #if ticker not in ['ASML', 'TEAM', 'BIDU', 'CHKP', 'FOXA', 'JD', 'NTES', 'PDD', 'TCOM']:
    insider_data = pd.read_csv(DATA_LOCATION_INSIDER_PROCESSED + ticker + '.csv', index_col=0)
    insider = pd.concat([insider,insider_data])

# Save to file
insider.to_csv(DATA_LOCATION_INSIDER_PROCESSED + 'all_concatenated.csv')

#Show content
insider

VCXA
TXG
YI
YQ
ATNF
FLWS
BCOW
ONEM
FCCY
SRCE
DIBS
ME
ADER
TSVT
TWOU
QFIN
KRKR
FDMT
nan
FVAM
JOBS
nan
SVNA
VII
ETNB
LAX
CAFD
NMTR
MASS
JFU
AVHI
AZ
AAON
ABAX
ABCM
ABCL
ABEO
ABGI
ABILF
ABMD
nan
AXAS
ASPA
ABSI
ACIU
ACIA
ACTG
ASO
ACHC
ACAD
ACST
AXDX
XLRN
ANCX
ACCD
ENER
ARAY
ACEV
nan
ACRX
ACER
ACETQ
AKAOQ
nan
ACHV
ACHL
ACHN
ACIW
ACRS
ACMR
ACNB
ACOR
ATVI
ACTA
ABOS
ACXP
AFIB
ACVA
ADAG
ADGI
ADMS
ADMP
AHCO
ADAP
ADPT
ADXN
ADUS
AEY
IOTS
ADIL
ACET
nan
ADMA
ADBE
ADSE
ADTH
ADTN
AAAP
ADES
AEIS
nan
AMD
ADV
CXRXF
ADXS
ADN
ADVM
ADYX
AEGN
AGLE
AEHR
AMTX
ARBG
AERI
AERC
AVTE
AVAV
ASLE
nan
AIH
AEZS
nan
AEMD
nan
LIDR
AFAQ
AFCG
AFMD
AFBI
AFRM
AFYA
AGBA
AGEN
AGGR
AGRX
AGIL
AGYS
AGIO
AGMH
AGNC
API
RICO
nan
AGFY
AGFS
AHRN
nan
AIKI
ALRN
AIMT
AIRT
ATSG
ABNB
AIRG
ANTE
AIRS
AKAM
nan
AKTX
AKCA
AKBA
KERN
AKRO
AKRXQ
AKUS
AKTS
AKYA
AHG
AKU
ALRM
ALSK
TCRT
ALAC
ALBO
ALDR
ALDX
ALEC
ALR
ALXN
ALF
ASTL
ALCO
ALGN
ALHC
ALGS
ALIM
ALYA
ALJJ
ALKT
ALKS
ALLK
ALLR
BIRD
AMOT
ABTX
ALGT
ALGM
ALNA
AHGP
ARLP
AESE
AHPI
ALLO
ALLT
ALVR
AFAM
A

,X,FilingDate,TradeDate,Ticker,Insider Name,Title,Trade Type,Price,Qty,Owned,delta_Own,Value,1d,1w,1m,6m,ticker,InsiderName,TradeType,CompanyName
0,D,2022-02-24 19:40:46,2022-02-22,TXG,NaN,See Remarks,NaN,7859.0,-360,160326,0%,-28294.0,NaN,NaN,NaN,NaN,TXG,Hindson Benjamin J.,S - Sale+OE,NaN
1,NaN,2022-02-24 19:39:35,2022-02-22,TXG,NaN,CEO,NaN,7860.0,-551,1006840,0%,-43309.0,NaN,NaN,NaN,NaN,TXG,Saxonov Serge,S - Sale,NaN
2,NaN,2022-02-24 19:38:29,2022-02-22,TXG,NaN,CFO,NaN,7860.0,-288,38531,-1%,-22637.0,NaN,NaN,NaN,NaN,TXG,McAnear Justin J.,S - Sale,NaN
3,NaN,2022-02-24 19:37:57,2022-02-22,TXG,NaN,Chief Commercial Officer,NaN,7860.0,-288,97365,0%,-22637.0,NaN,NaN,NaN,NaN,TXG,Crutchfield Bradford,S - Sale,NaN
4,D,2022-02-17 16:48:10,2022-02-15,TXG,NaN,Chief Commercial Officer,NaN,9513.0,-17000,97653,-15%,-1617261.0,NaN,NaN,NaN,NaN,TXG,Crutchfield Bradford,S - Sale+OE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,NaN,2018-05-29 16:08:26,2018-05-24,ZNGA,NaN,Dir,NaN,411.0,-12000,337529,-3%,-49320.0,NaN,NaN,NaN,NaN,ZNGA,Siminoff Ellen F,S - Sale,NaN
202,NaN,2018-04-24 17:14:36,2018-04-24,ZNGA,NaN,Dir,NaN,352.0,-12000,332107,-3%,-42240.0,NaN,NaN,NaN,NaN,ZNGA,Siminoff Ellen F,S - Sale,NaN
203,NaN,2018-04-19 16:10:36,2018-04-18,ZNGA,NaN,Pres of Publishing,NaN,357.0,-20000,391225,-5%,-71400.0,NaN,NaN,NaN,NaN,ZNGA,Kim Bernard Jin,S - Sale,NaN
204,NaN,2018-04-19 16:08:41,2018-04-18,ZNGA,NaN,COO,NaN,357.0,-20000,340056,-6%,-71400.0,NaN,NaN,NaN,NaN,ZNGA,Bromberg Matthew S,S - Sale,NaN


In [13]:
insider.groupby('delta_Own').X.count()

delta_Own
+1%      1596
+10%      133
+100%      25
+101%       3
+102%       1
         ... 
-98%       93
-99%      118
0%       6063
>999%      61
New       537
Name: X, Length: 691, dtype: int64

In [14]:
insider.groupby('TradeType').X.count()

TradeType
P - Purchase     9811
S - Sale        12329
S - Sale+OE     35055
Name: X, dtype: int64

### Combine transactions by same person at same day
This is needed for step 3

## Time Series Stock prices
Again, figure out the various parameters

### Constants
Defining the various things we are interested in

In [15]:
# Period of interest
end_time = datetime.datetime.now()
end_time = datetime.datetime(2021,12,31,23,59,59)
end_time_unix = int(time.mktime(end_time.timetuple()))

start_time = end_time - relativedelta(years=4)
start_time = datetime.datetime(2018,1,1,0,0,0)
start_time_unix = int(time.mktime(start_time.timetuple()))

### Download timeseries
Loop through all tickers and download corresponding CSVs.

In [16]:
if REDOWNLOAD_TIMESERIES_DATA:
    for ticker in tickers:
        ticker = ticker.replace(' ', '+')
        print(ticker)
        url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={start_time_unix}&period2={end_time_unix}&interval=1d&events=history&includeAdjustedClose=true'
        timeseries = pd.read_csv(url)
        timeseries['ticker'] = ticker
        # Nothing to do here
        #timeseries.to_csv(DATA_LOCATION_TIME_SERIES_RAW + ticker + '.csv')
        timeseries.to_csv(DATA_LOCATION_TIME_SERIES_PROCESSED + ticker + '.csv')        

VCXA
TXG
YI
YQ
ATNF
FLWS
BCOW
ONEM
FCCY


HTTPError: HTTP Error 404: Not Found

### Combine timeseries to a single table

In [ ]:
timeseries = pd.DataFrame()
dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d')

for ticker in tickers:
    ts_data = pd.read_csv(DATA_LOCATION_TIME_SERIES_PROCESSED + ticker + '.csv', index_col=0, parse_dates=['Date'], date_parser=dateparse)
    timeseries = pd.concat([timeseries,ts_data])

### Add Information

In [ ]:
# Calculate Marketcap
timeseries['MarketCap'] = timeseries.Open * timeseries.Volume
#Volatility - Figure this out!
timeseries['prev_close'] = timeseries.groupby('ticker')['Close'].shift()
timeseries['Returns'] = (timeseries.Close / timeseries.prev_close)-1
timeseries = timeseries.drop('prev_close', 1)

# Export
timeseries.to_csv(DATA_LOCATION_TIME_SERIES_PROCESSED + 'all_concatenated.csv')

# Show table
timeseries

### Playing around

#### Visualize price movement for a single stock
This be interactive

In [ ]:
visualize_price_moving_average('TSLA', 20,50)
visualize_price_bollinger_bands('TSLA')

In [ ]:
visualize_price_moving_average('AAPL', 20,50)
visualize_price_bollinger_bands('AAPL')

### Compare pairwise growth
Showing the five first ticks, as clutter and performance

In [ ]:
ts_Close_pivot = timeseries[['Date', 'Close', 'ticker']].pivot(index=['Date'], columns=['ticker'], values='Close')
ts_Close_pivot

In [ ]:
sns.pairplot(ts_Close_pivot.iloc[:, 0:5], diag_kind="kde")
plt.show()

### Visualize Volatility

In [ ]:
ts_Returns_pivot = timeseries[['Date', 'Returns', 'ticker']].pivot(index=['Date'], columns=['ticker'], values='Returns')
ts_Returns_pivot

In [ ]:
sns.histplot(data=ts_Returns_pivot.iloc[:, 0:5], kde=False, element='step', fill=False)

#### Marketcap
Showing marketcap evolution over time - Should be skipped, as this is waaayy too cluttered

In [ ]:
sns.lineplot(data=timeseries, x="Date", y="MarketCap", hue='ticker')